In [1]:
import pandas as pd
import joblib
import numpy as np

In [2]:
X_test = pd.read_csv('../data/X_test_balanceado.csv')

In [3]:
# Load the scaler object from the file
scaler = joblib.load('scaler.pkl')

In [4]:
# Scale the test data using the saved scaler object
X_test_scaled = scaler.transform(X_test)

In [5]:
test_data = X_test_scaled[0]

In [6]:
test_data.shape

(777,)

In [11]:
test_data = test_data.reshape(1, 777)

In [12]:
import json
import requests

# Define the URL for the prediction request
url = 'http://localhost:7000/v1/models/bm:predict'

# Define the input data for the prediction request
input_data = test_data.astype('float32').tolist()
request_data = {'instances': input_data}

# Send the prediction request to the API
response = requests.post(url, data=json.dumps(request_data))


# Parse the prediction result from the response
result = json.loads(response.text)['predictions'][0]
result = np.array(result).astype('float32')


# Print the prediction result
print(result)

[0.56865275]


In [13]:
import json
import requests
import numpy as np
import time
import multiprocessing

# Define the URL for the prediction request
url = 'http://localhost:7000/v1/models/bm:predict'

# Define the input data for the prediction request
input_data = np.random.rand(1, 777).astype('float32').tolist()
request_data = {'instances': input_data}

# Define a function that sends a prediction request to the server
def send_request(url, request_data, queue):
    start_time = time.time()
    response = requests.post(url, data=json.dumps(request_data))
    end_time = time.time()
    result = json.loads(response.text)['predictions'][0]
    result = np.array(result).astype('float32')
    queue.put((result, end_time - start_time))

# Define the number of processes to spawn
num_processes = 10

# Define the number of requests per process
num_requests = 5000

# Define the total time to run the stress test
total_time = 45

# Compute the delay between requests
delay = total_time / (num_processes * num_requests)

# Spawn the processes
processes = []
queue = multiprocessing.Queue()
for i in range(num_processes):
    p = multiprocessing.Process(target=send_request, args=(url, request_data, queue))
    processes.append(p)
    p.start()

# Collect the results from the processes
results = []
while len(results) < num_processes * num_requests:
    try:
        result = queue.get(timeout=1)
        results.append(result)
    except:
        pass

# Compute the percentage of requests that were responded
num_responded = len(results)
num_total = num_processes * num_requests
percent_responded = num_responded / num_total * 100

# Compute the average processing time
avg_time = sum([r[1] for r in results]) / num_responded

# Print the results
print(f'Percentage of requests responded: {percent_responded:.2f}%')
print(f'Average processing time: {avg_time:.2f} seconds')